In [1]:
import pandas as pd
import numpy as np
print('Libraries loaded.')

Libraries loaded.


Load the CSV file created from last notebook which contains Canada postal codes. borough and neighbourhood information.

In [2]:
df = pd.read_csv('CA_PC.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Geocoder package was tried and couldn't work properly.
Use Nominatim from geopy.geocoders instead to scrap latitude and longtitude data for each postal code:

In [3]:
import geocoder
from geopy.geocoders import Nominatim

CLIENT_ID = 'C3UOHBNDZTAKKMLLAXEZKBOSFCLEZAT5J5UUJ3XUJ5OEUZJB' # your Foursquare ID
CLIENT_SECRET = '4TCGSEHFSEHYRTFINMQHYC3BW10230XA153FKOD3LGS5EP2L' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C3UOHBNDZTAKKMLLAXEZKBOSFCLEZAT5J5UUJ3XUJ5OEUZJB
CLIENT_SECRET:4TCGSEHFSEHYRTFINMQHYC3BW10230XA153FKOD3LGS5EP2L


In [15]:
dfn = df.copy()
dfn['Neighbourhood'] = dfn.Neighbourhood.str.split(',')
proxies = {'http': 'WEBPROXY.WATSON.IBM.COM:8080', 'https': 'WEBPROXY.WATSON.IBM.COM:8080', 'socks':'WEBPROXY.emea.IBM.COM:8080'}

for i in df.index:
    address = '{}, Toronto, Ontario, {}, Canada'.format(dfn.loc[i, 'Neighbourhood'][0], df.loc[i, 'Postal Code'])
    geolocator = Nominatim(user_agent="foursquare_agent", proxies=proxies, timeout=20)
    location = geolocator.geocode(address)
    if location == None:
        print(i)
    else:
        latitude = location.latitude
        longitude = location.longitude
        df.loc[i, 'Latitude'] = latitude
        df.loc[i, 'Longitude'] = longitude
        print(location)

df.info()

Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 1Z5, Canada
1
2
3
Queen's Park, University Avenue, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5S 3M2, Canada
5
Malvern, McLevin Avenue, Browns Corners, Scarborough North, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 2L2, Canada
7
8
9
10
11
Rouge Hill, Lawrence Avenue East, West Rouge, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1C 3Y1, Canada
13
14
15
16
Eringate, Etobicoke, Etobicoke Centre, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9C 3Z7, Canada
18
19
Berczy Park, King East, Toronto, Golden Horseshoe, Ontario, Canada
21
Woburn, Scarborough—Guildwood, Scarborough, Toronto, Golden Horseshoe, Ontario, M1H 2A2, Canada
23
24
25
26
27
28
29
Richmond-Adelaide Centre, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5H, Canada
31
32
Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada
34
35
Harb

As we can see from the output above, geopy couldn't get all geographical coordinates we wanted for all 103 postal codes. Therefore, I have to use the csv file provided to continue with the project.
Load latitude and lontitude csv file as Pandas dataframe and concat with our postal code dataframe:

In [16]:
geo_data = pd.read_csv('Geospatial_Coordinates.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
for i in df.index:
    for j in geo_data.index:
        if geo_data['Postal Code'][j] == df['Postal Code'][i]:
            df.loc[i, 'Latitude'] = geo_data.loc[j, 'Latitude']
            df.loc[i, 'Longitude'] = geo_data.loc[j, 'Longitude']
df.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [19]:
df.to_csv('CA_PC_w_latlon.csv')